In [1]:
import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Funciones auxiliares

In [2]:
def load_kdd_dataset(data_path):
    """Lectura del conjunto de datos NSL-KDD."""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
    attributes = [attr[0] for attr in dataset["attributes"]]
    return pd.DataFrame(dataset["data"], columns=attributes)

In [3]:
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

## 1. Lectura del conjunto de datos

In [ ]:
df = load_kdd_dataset("datasets/NSL-KDD/KDDTrain+.arff")

In [ ]:
df

## 2. División del conjunto de datos

In [ ]:
train_set, val_set, test_set = train_val_test_split(df, stratify='protocol_type')

In [ ]:
print("Longitud del Training Set:", len(train_set))
print("Longitud del Validation Set:", len(val_set))
print("Longitud del Test Set:", len(test_set))

## 3 Limpiando los datos

In [ ]:
# Separamos las características de entrada de la característica de salida
X_train = train_set.drop("class", axis=1)
y_train = train_set["class"].copy()

In [ ]:
# Para ilustrar esta sección vamos a añadir algunos valores nulos 
# a algunas características del conjunto de datos
X_train.loc[(X_train["src_bytes"]>400) & (X_train["src_bytes"]<800), "src_bytes"] = np.nan
X_train.loc[(X_train["dst_bytes"]>500) & (X_train["dst_bytes"]<2000), "dst_bytes"] = np.nan
X_train

In [ ]:
# Comprobamos si existe algún atributo con valores nulos
X_train.isna().any()

In [ ]:
# Seleccionamos las filas que contienen valores nulos
filas_valores_nulos  = X_train[X_train.isnull().any(axis=1)]
filas_valores_nulos

### Opción 1: Eliminamos las filas con valores nulos

In [ ]:
# Copiamos el conjunto de datos para no alterar el original
X_train_copy = X_train.copy()

In [ ]:
# Eliminamos las filas con valores nulos
X_train_copy.dropna(subset=["src_bytes", "dst_bytes"], inplace=True)
X_train_copy

In [ ]:
# Contamos el número de filas eliminadas
print("El número de filas eliminadas es:", len(X_train) - len(X_train_copy))

### Opción 2: Eliminamos los atributos con valores nulos

In [ ]:
# Copiamos el conjunto de datos para no alterar el original
X_train_copy = X_train.copy()

In [ ]:
# Eliminamos los atributos con valores nulos
X_train_copy.drop(["src_bytes", "dst_bytes"], axis=1, inplace=True)
X_train_copy

In [ ]:
# Contamos el número de atributos eliminados
print("El número de atributos eliminados es:", len(list(X_train)) - len(list(X_train_copy)))

### Opción 3: Rellenamos los valores nulos con un valor determinado

In [ ]:
# Copiamos el conjunto de datos para no alterar el original
X_train_copy = X_train.copy()

In [ ]:
# Rellenamos los valores nulos con la media de los valores del atributo
media_srcbytes = X_train_copy["src_bytes"].mean()
media_dstbytes = X_train_copy["dst_bytes"].mean()

X_train_copy["src_bytes"] = X_train_copy["src_bytes"].fillna(media_srcbytes)
X_train_copy["dst_bytes"] = X_train_copy["dst_bytes"].fillna(media_dstbytes)

X_train_copy

In [ ]:
# Copiamos el conjunto de datos para no alterar el original
X_train_copy = X_train.copy()

In [ ]:
# Un valor muy alto en el atributo puede disparar la media
# Rellenamos los valores con la mediana
mediana_srcbytes = X_train_copy["src_bytes"].median()
mediana_dstbytes = X_train_copy["dst_bytes"].median()

X_train_copy["src_bytes"] = X_train_copy["src_bytes"].fillna(mediana_srcbytes)
X_train_copy["dst_bytes"] = X_train_copy["dst_bytes"].fillna(mediana_dstbytes)

X_train_copy

### Existe otra alternativa para la opción 3 que consiste en usar la clase Imputer de sklearn

In [ ]:
# Copiamos el conjunto de datos para no alterar el original
X_train_copy = X_train.copy()

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
# La clase imputer no admite valores categoricos, eliminamos los atributos categoricos
X_train_copy_num = X_train_copy.select_dtypes(exclude=['object'])
X_train_copy_num.info()

In [ ]:
# Se le proporcionan los atributos numericos para que calcule los valores
imputer.fit(X_train_copy_num)

In [ ]:
# Rellenamos los valores nulos
X_train_copy_num_nonan = imputer.transform(X_train_copy_num)

In [ ]:
# Transformamos el resultado a un DataFrame de Pandas
X_train_copy = pd.DataFrame(X_train_copy_num_nonan, columns=X_train_copy_num.columns)

In [ ]:
X_train_copy.head(10)

## 4. Transformación de atributos categóricos a numéricos

In [ ]:
X_train = train_set.drop("class", axis=1)
y_train = train_set["class"].copy()

In [ ]:
X_train.info()

In [ ]:
protocol_type = X_train['protocol_type']
protocol_type_encoded, categorias = protocol_type.factorize()

In [ ]:
# Mostramos por pantalla como se han codificado
for i in range(10):
    print(protocol_type.iloc[i], "=", protocol_type_encoded[i])

In [ ]:
print(categorias)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

protocol_type = X_train[['protocol_type']]

ordinal_encoder = OrdinalEncoder()
protocol_type_encoded = ordinal_encoder.fit_transform(protocol_type)

In [ ]:
# Mostramos por pantalla como se han codificado
for i in range(10):
    print(protocol_type["protocol_type"].iloc[i], "=", protocol_type_encoded[i])

In [ ]:
print(ordinal_encoder.categories_)

In [ ]:
# La sparse matrix solo almacena la posicion de los valores que no son '0' para ahorrar memoria
from sklearn.preprocessing import OneHotEncoder

protocol_type = X_train[['protocol_type']]

oh_encoder = OneHotEncoder()
protocol_type_oh = oh_encoder.fit_transform(protocol_type)
protocol_type_oh

In [ ]:
# Convertir la sparse matrix a un array de Numpy
protocol_type_oh.toarray()

In [ ]:
# Mostramos por pantalla como se han codificado
for i in range(10):
    print(protocol_type["protocol_type"].iloc[i], "=", protocol_type_oh.toarray()[i])

In [ ]:
print(ordinal_encoder.categories_)

In [ ]:
oh_encoder = OneHotEncoder(handle_unknown='ignore')

In [ ]:
pd.get_dummies(X_train['protocol_type'])

## 5. Escalado del conjunto de datos

In [ ]:
X_train = train_set.drop("class", axis=1)
y_train = train_set["class"].copy()

In [ ]:
from sklearn.preprocessing import RobustScaler

scale_attrs = X_train[['src_bytes', 'dst_bytes']]

robust_scaler = RobustScaler()
X_train_scaled = robust_scaler.fit_transform(scale_attrs)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=['src_bytes', 'dst_bytes'])

In [ ]:
X_train_scaled.head(10)

In [ ]:
X_train.head(10)